# Python Data Engineer Code Challenge

[Descripción]

In [2]:
import csv
import pandas as pd
import json
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine, Column, Integer, String, Boolean, Date
from sqlalchemy import func, text

In [7]:
csv_file = "data/candidates.csv"
data = pd.read_csv(csv_file, sep=";")
print(data[0:5])
print(len(data))

   First Name   Last Name                      Email Application Date  \
0  Bernadette   Langworth        leonard91@yahoo.com       2021-02-26   
1      Camryn    Reynolds        zelda56@hotmail.com       2021-09-09   
2       Larue      Spinka   okey_schultz41@gmail.com       2020-04-14   
3        Arch      Spinka     elvera_kulas@yahoo.com       2020-10-01   
4       Larue  Altenwerth  minnie.gislason@gmail.com       2020-05-20   

   Country  YOE  Seniority                         Technology  \
0   Norway    2     Intern                      Data Engineer   
1   Panama   10     Intern                      Data Engineer   
2  Belarus    4  Mid-Level                     Client Success   
3  Eritrea   25    Trainee                          QA Manual   
4  Myanmar   13  Mid-Level  Social Media Community Management   

   Code Challenge Score  Technical Interview Score  
0                     3                          3  
1                     2                         10  
2          

We can tell that there's not a column that tells us whether the candidate was hired or not, however, we have the right information to tell. In order to make future queries easier, we are going to create a new column named 'hired' later.

### Step II: Establishing a connection to the database and creating the table
NOTE: For this step, it's necessary to have previously created a file with your database credentials. In this case, this file is called db_config.json and contains the username, password, host, port and database name.

In [46]:
# establecer la conexión
with open('./db_config.json', 'r') as file:
    data = json.load(file)

engine = create_engine(f'mysql://{data["user"]}:{data["password"]}@{data["host"]}:{data["server"]}/{data["db"]}')

# Para que una clase pueda considerarse un modelo, debe heredar de otra clase, en este caso la clase Base
Base = declarative_base()

# Crear el modelo
class Candidate(Base):
    __tablename__ = "candidates" # atributo tablename -> indicar el nombre de la tabla

    id = Column(Integer, primary_key=True, autoincrement=True)
    first_name = Column(String(50), nullable=False)
    last_name = Column(String(50), nullable=False)
    email = Column(String(50), nullable=False)
    app_date = Column(Date, nullable=False)
    country = Column(String(60), nullable=False)
    yoe = Column(Integer, nullable=False)
    seniority = Column(String(20), nullable=False)
    technology = Column(String(50), nullable=False)
    cc_score = Column(Integer, nullable=False)
    ti_score = Column(Integer, nullable=False)
    hired = Column(Boolean, nullable=False)

# crear la tabla
Base.metadata.create_all(bind=engine)


C:\Users\Mrìa Angelica\AppData\Local\Temp\ipykernel_9532\2096287153.py:8: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


### Step III: Modifying and inserting data into the table
First, we are going to change the names of the columns in the dataframe to match the names of the columns in the table, then we are going to create the column "hired". 

NOTE: A candidate is considered HIRED when she or he has both scores greater than or equal to 7.

In [47]:
file = "data/candidates.csv"
df = pd.read_csv(file, sep=";")

df.columns = ["first_name","last_name","email","app_date","country","yoe","seniority","technology","cc_score","ti_score"]

df['hired'] = ((df['cc_score'] >= 7) & (df['ti_score'] >= 7)).astype(int)
print(df[0:3])

df.to_sql(con=engine,name=Candidate.__tablename__, if_exists='append', index=False)
# params: con=engine connected to the db, name=table name to insert, if_exists=if the table exists, insert the data, index=False= remove the default index of pandas dfs.

   first_name  last_name                     email    app_date  country  yoe  \
0  Bernadette  Langworth       leonard91@yahoo.com  2021-02-26   Norway    2   
1      Camryn   Reynolds       zelda56@hotmail.com  2021-09-09   Panama   10   
2       Larue     Spinka  okey_schultz41@gmail.com  2020-04-14  Belarus    4   

   seniority      technology  cc_score  ti_score  hired  
0     Intern   Data Engineer         3         3      0  
1     Intern   Data Engineer         2        10      0  
2  Mid-Level  Client Success        10         9      1  


50000